In [1172]:
import gensim
from gensim import corpora
import multiprocessing
from gensim.models import KeyedVectors
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import pkg_resources
import pandas as pd
import numpy as np

In [1173]:
import pandas as pd
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from gensim.parsing.preprocessing import preprocess_string
import random
import warnings
warnings.filterwarnings("ignore")

In [1174]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [1908]:
df = pd.read_csv('tweets_processed.csv', usecols=['clean_keyword', 'clean_location'])
df2 = pd.read_csv('tweets_update.csv', usecols=['denoised_text', 'lem_tokes', 'lem_adv'])
df['lem_v'] = df2['lem_tokes']
df['lem_av'] = df2['lem_adv']
df['tweets'] = df2['denoised_text']

In [1909]:
df

,clean_keyword,clean_location,lem_v,lem_av,tweets
0,NaN,NaN,our deeds be the reason of this earthquake may...,our deeds are the reason of this earthquake ma...,our deeds are the reason of this earthquake m...
1,NaN,NaN,forest fire near la ronge sask canada,forest fire near la ronge sask canada,forest fire near la ronge sask canada
2,NaN,NaN,all residents ask to shelter in place be be no...,all residents asked to shelter in place are be...,all residents asked to shelter in place are b...
3,NaN,NaN,people receive wildfires evacuation order in c...,people receive wildfires evacuation orders in ...,people receive wildfires evacuation orders in...
4,NaN,NaN,just get send this photo from ruby alaska as s...,just got sent this photo from ruby alaska as s...,just got sent this photo from ruby alaska as ...
...,...,...,...,...,...
10871,NaN,NaN,earthquake safety los angeles safety fasteners...,earthquake safety los angeles safety fasteners...,earthquake safety los angeles safety fasteners...
10872,NaN,NaN,storm in rhode island worse than last hurrican...,storm in rhode island worse than last hurrican...,storm in rhode island worse than last hurrica...
10873,NaN,NaN,green line derailment in chicago,green line derailment in chicago,green line derailment in chicago
10874,NaN,NaN,meg issue hazardous weather outlook hwo,meg issues hazardous weather outlook hwo,meg issues hazardous weather outlook hwo


In [1910]:
target_df = pd.read_csv('train.csv', usecols=['target'])
target = list(target_df.target)

In [1911]:
target_df.target.isna().sum()

0

In [1912]:
train_len = len(target)

In [1913]:
train_df, test_df = df.iloc[:train_len, :], df.iloc[train_len:, :] 
train_df['target'] = target
train_df.drop_duplicates(subset=['tweets'], keep='last', inplace=True)

In [1914]:
len(train_df)

6830

In [1915]:
len(test_df)

3263

In [1916]:
tweets = list(train_df.lem_av)
for x in test_df.lem_av:
    tweets.append(x) 
len(tweets)

10093

In [1927]:
def kw_loc_column_fix(df):
    df['clean_keyword'] = df.clean_keyword.astype(str)
    df['clean_keyword'] = df.clean_keyword.str.lower()
    df['clean_keyword'] = df.clean_keyword.str.replace(r'\bnan\b', r'')
    #df['clean_keyword'] = df.clean_keyword.str.replace(r' ', r'_')
    df['clean_keyword'] = df.clean_keyword.str.strip()
    df['clean_location'] = df.clean_location.astype(str)
    df['clean_location'] = df.clean_location.str.lower()
    df['clean_location'] = df.clean_location.str.strip()
    df['clean_location'] = df.clean_location.str.replace(r'\bnan\b', r'')
   # df['clean_location'] = df.clean_location.str.replace(r' ', r'_')
    return df

In [1942]:
df = kw_loc_column_fix(df)

In [1943]:
train_df = kw_loc_column_fix(train_df)
test_df = kw_loc_column_fix(test_df)

In [1944]:
keyword = pd.get_dummies(train_df.clean_keyword, prefix='keyword')
location = pd.get_dummies(train_df.clean_location, prefix='location')
train_df_secondary = pd.concat([keyword, location], axis=1)

keyword_test = pd.get_dummies(test_df.clean_keyword, prefix='keyword')
location_test = pd.get_dummies(test_df.clean_location, prefix='location')
test_df_secondary = pd.concat([keyword_test, location_test], axis=1)

In [1948]:
train_df['all_text'] = train_df[['lem_av', 'clean_location', 'clean_keyword']].agg(' '.join, axis=1)
test_df['all_text'] = test_df[['lem_av', 'clean_location', 'clean_keyword']].agg(' '.join, axis=1)

In [1949]:
tweets = list(train_df.all_text)
for x in test_df.all_text:
    tweets.append(x) 
len(tweets)

10093

## extract bigrams

In [1950]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
from nltk.corpus import stopwords

In [1952]:

def get_documents(text):
    docs = [row.split() for row in text]
    return docs

doc = get_documents(tweets)

def sentence_to_bi_grams(sentence):
    return ' '.join(phrases[sentence])

phrases = Phrases(doc, min_count=2, threshold=.4, connector_words=ENGLISH_CONNECTOR_WORDS)

frozen_phrases = phrases.freeze()

bigrams = []
for row in doc: 
    parsed_sentence = sentence_to_bi_grams(row)
    bigrams.append(parsed_sentence)
    
bigrams

['our deeds are the reason of this earthquake may_allah forgive us_all',
 'forest_fire near la ronge sask canada',
 'all residents asked to shelter in place are_being notified by officers no other evacuation or shelter in place orders are expected',
 'people receive wildfires evacuation orders in california',
 'just_got sent this photo from ruby alaska as smoke from wildfires pours into school',
 'rocky_fire update california highway closed in both directions due to lake county_fire afire wildfires',
 'flood_disaster heavy_rain causes flash_flooding of streets in manitou colorado springs areas',
 'am on top of the hill and can_see fire in the woods',
 'there_is an emergency evacuation happening_now in the building across_the_street',
 'am_afraid that the tornado is_coming to our area',
 'three_people died from the heat_wave so_far',
 'haha south tampa is_getting flooded hah wait second live in south tampa what_am gon_na do what_am gon_na do fuck flooding',
 'raining flooding florida ta

In [1953]:
new_df = pd.DataFrame()
new_df['tweets'] = tweets
new_df['bigrams'] = bigrams

## prep text for vectorization

In [1954]:
import re

import nltk
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if word not in stopword_list and len(word) > 1:
                tokens.append(word.lower())
    return tokens

train_tweets = train_df.stems.astype(str).apply(tokenize_text)
test_tweets = test_df.stems.apply(tokenize_text)

In [1955]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            tokens.append(word.lower())
    return tokens

In [1956]:
texts = new_df.tweets.apply(tokenize_text)
bigram_tokes = new_df.bigrams.apply(tokenize_text)

In [1957]:
train_df['texts'] = list(texts[0:6830])
test_df['texts'] = list(texts[6830:])
train_df['bg_texts'] = list(bigram_tokes[0:6830])
test_df['bg_texts'] = list(bigram_tokes[6830:])

In [1959]:
len(train_df)

6830

In [1960]:
test_df['length'] = test_df.texts.str.len()
test_df.loc[test_df['length'] == test_df['length'].min()]

,clean_keyword,clean_location,lem_v,lem_av,tweets,all_text,texts,bg_texts,length
7627,,,awesome,awesome,awesome,awesome,[awesome],[awesome],1
7667,,,aftershock,aftershock,aftershock,aftershock,[aftershock],[aftershock],1
8855,,,displace,displaced,displaced,displaced,[displaced],[displaced],1
9167,,,fatality,fatality,fatality,fatality,[fatality],[fatality],1
9171,,,fatality,fatality,fatality,fatality,[fatality],[fatality],1
9174,,,fatality,fatality,fatality,fatality,[fatality],[fatality],1
9992,,,rainstorm,rainstorm,rainstorm,rainstorm,[rainstorm],[rainstorm],1
10180,,,scream,screams,screams,screams,[screams],[screams],1
10184,,,scream,screams,screams,screams,[screams],[screams],1


In [1962]:
train_df['length'] = train_df.texts.str.len()

train_df.loc[train_df['length'] == train_df['length'].min()]

,clean_keyword,clean_location,lem_v,lem_av,tweets,target,all_text,texts,bg_texts,length
28,,,cool,cool,cool,0,cool,[cool],[cool],1


In [1963]:
X_train, X_test, y_train, y_test = train_test_split(train_df.bg_texts, train_df.target, test_size=0.1, random_state=42)

In [1964]:
from collections import Counter
from statistics import mean

word_dict = Counter(X_train.sum())
# count number of words in corpus
num_words = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words} words in the training dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
vocab_size = len(word_count_sorted)
print(f'There are {vocab_size} unique words in the training dataset')
print("max len of training tweets",max([len(x) for x in X_train]))
print("max len of test tweets",max([len(x) for x in X_test]))
print("min len of training tweets",min([len(x) for x in X_train]))
print("min len of test tweets",min([len(x) for x in X_test]))
print("mean len of training tweets", mean([len(x) for x in X_train]))
print("mean len of test tweets",mean([len(x) for x in X_test]))

There are 82765 words in the training dataset
There are 18128 unique words in the training dataset
max len of training tweets 32
max len of test tweets 30
min len of training tweets 1
min len of test tweets 1
mean len of training tweets 13.464291524320807
mean len of test tweets 13.184480234260615


In [1965]:
def get_embedding(word, model, vector_size):
    if word in model.wv.vocab:
        return model[x]
    else: 
        return np.zeros(vector_size)

def get_average_vector(model, list_of_tweets):    
    vec_dicts = {}
    for tweet in list_of_tweets:
        average_vector = (np.mean(np.array([get_embedding(tweet, model)]), axis=0))
        v_dict = {tweet : (average_vector)}
        vec_dicts.update(v_dict)
        return vec_dicts

In [1966]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np     
import plotly

In [1967]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=42)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly



## uninitialized skipgram

In [1968]:
cores = multiprocessing.cpu_count()

In [1985]:
# uninitializeed model
model_ui_sg = Word2Vec(
        vector_size=100,
        sg=1,
        negative=6,
        ns_exponent=0.35,
        min_count=4,
        window=3,
        workers=cores)
vocab = model_ui_sg.build_vocab(X_train)

model_ui_sg.train(X_train, total_examples=len(X_train), total_words=num_words, compute_loss=True, epochs=10)

(531533, 827650)

In [1986]:
model_ui_sg.wv.most_similar(positive="earthquake")

[('earth', 0.9946778416633606),
 ('washington', 0.9936268329620361),
 ('australia', 0.9932706356048584),
 ('worst', 0.9932101964950562),
 ('western', 0.9931592345237732),
 ('area', 0.993135392665863),
 ('highway', 0.9930528998374939),
 ('history', 0.9930253624916077),
 ('refugees', 0.9929182529449463),
 ('during', 0.9929112792015076)]

In [1987]:
x_vals, y_vals, labels = reduce_dimensions(model_ui_sg)
plot_function(x_vals, y_vals, labels)

## self initialized skipgram

In [1972]:
# uninitializeed model
model_si_sg = Word2Vec(
        sentences=X_train,
        vector_size=100,
        sg=0,
        min_count=4,
        window=3,
        workers=cores)
vocab = model_si_sg.build_vocab(X_train)

model_si_sg.train(X_train, total_words=num_words, compute_loss=True, epochs=20)

(1062985, 1655300)

In [1973]:
model_si_sg.wv.most_similar(positive="earthquake")

[('global', 0.9784128665924072),
 ('volcano', 0.9779451489448547),
 ('nigeria', 0.9778736233711243),
 ('south', 0.9778392910957336),
 ('bombing', 0.9771696925163269),
 ('utah', 0.9766325950622559),
 ('northern_california', 0.976618766784668),
 ('homes', 0.9764298796653748),
 ('coordinated_universal', 0.9762946367263794),
 ('north', 0.9762483835220337)]

In [1974]:
model_si_sg.wv.most_similar(positive="fire")

[('fire_truck', 0.8825393319129944),
 ('road', 0.8793429732322693),
 ('on', 0.8761358857154846),
 ('two_buildings', 0.8701041340827942),
 ('live', 0.8675208687782288),
 ('fatalities', 0.8673534989356995),
 ('car', 0.8669477105140686),
 ('apartment_buildings', 0.8648421168327332),
 ('buildings', 0.8646915555000305),
 ('new', 0.8630614876747131)]

In [1975]:
x_vals, y_vals, labels = reduce_dimensions(model_si_sg)
plot_function(x_vals, y_vals, labels)

# cbow

In [1976]:
# uninitializeed model
model_ui_bow = Word2Vec(
        vector_size=200,
        sg=0,
        #sample=0,
        #negative=11,
        #ns_exponent=0.3,
        cbow_mean=0,

        min_count=4,
        window=3, 
        workers=cores)
vocab = model_ui_bow.build_vocab(X_train)

model_ui_bow.train(X_train, total_examples=model_ui_bow.corpus_count, epochs=20)

(1063292, 1655300)

In [1977]:
model_ui_bow.wv.most_similar(positive="wildfire")

[('northern_california', 0.9947760701179504),
 ('homes_razed', 0.9864144325256348),
 ('australia', 0.985486626625061),
 ('oregon', 0.9854139685630798),
 ('photography', 0.9849956035614014),
 ('eyewitness', 0.9847315549850464),
 ('bridge_collapse', 0.9837499856948853),
 ('india', 0.9833371043205261),
 ('canada', 0.9832950830459595),
 ('economic', 0.9832637906074524)]

In [1978]:
model_ui_bow.get_latest_training_loss()

0.0

In [1979]:
x_vals, y_vals, labels = reduce_dimensions(model_ui_bow)
plot_function(x_vals, y_vals, labels)

In [546]:
##### model.save("w2v_ui.model")
model = Word2Vec.load("w2v_ui.model")
word_vectors = model_ui.wv
word_vectors.save("w2v_ui.wordvectors")

NameError: name 'model_ui' is not defined

In [1980]:
# self initialized model
model_si_bow = Word2Vec(
        X_train,
        vector_size=100, 
        cbow_mean=0,
        #sample=1e-5,
        min_count=5,
        window=2, 
        workers=cores)
vocab = model_si_bow.build_vocab(X_train)

model_si_bow.train(X_train,  total_examples=len(X_train), total_words=num_words, epochs=10)

(491884, 827650)

In [1981]:
model_si_bow.wv.most_similar(positive="wildfire")

[('colorado', 0.9960964918136597),
 ('canada', 0.9960824847221375),
 ('worldwide', 0.9960367679595947),
 ('snowstorm', 0.9956767559051514),
 ('york', 0.9956706762313843),
 ('alabama', 0.9956239461898804),
 ('manchester', 0.9956098794937134),
 ('washington', 0.9955883026123047),
 ('australia', 0.9955694079399109),
 ('t', 0.9954876899719238)]

In [1982]:
x_vals, y_vals, labels = reduce_dimensions(model_si_bow)
plot_function(x_vals, y_vals, labels)

## pretrained word_vectors model

In [1687]:
from nltk.tokenize import word_tokenize
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [1432]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath("C:\\Users\\root\\Projects\\nlp\\disaster_tweets\\utils\\glove.6B.100d.txt")
tmp_file = get_tmpfile("C:\\Users\\root\\Projects\\nlp\\disaster_tweets\\utils\\test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file, binary=False)

In [1993]:
train_df['tokens'] = train_df.lem_av.apply(tokenize_text)

In [2077]:
X_train, X_test, y_train, y_test = train_test_split(train_df, train_df.target, test_size=0.1, random_state=42)

In [2078]:
y_test

5198    0
2283    0
5082    1
5180    0
4812    1
       ..
4841    1
6514    1
4902    1
5550    0
3829    0
Name: target, Length: 683, dtype: int64

In [2079]:
from collections import Counter
from statistics import mean

word_dict = Counter(X_train.tokens.sum())
# count number of words in corpus
num_words = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words} words in the training dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
vocab_size = len(word_count_sorted)
print(f'There are {vocab_size} unique words in the training dataset')
print("max len of training tweets",max([len(x) for x in X_train.tokens]))
print("max len of test tweets",max([len(x) for x in X_test.tokens]))
print("min len of training tweets",min([len(x) for x in X_train.tokens]))
print("min len of test tweets",min([len(x) for x in X_test.tokens]))
print("mean len of training tweets", mean([len(x) for x in X_train.tokens]))
print("mean len of test tweets",mean([len(x) for x in X_test.tokens]))

There are 82872 words in the training dataset
There are 12808 unique words in the training dataset
max len of training tweets 30
max len of test tweets 31
min len of training tweets 1
min len of test tweets 1
mean len of training tweets 13.481698389458272
mean len of test tweets 13.175695461200586


In [2080]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('utils//glove.6B.100d.txt', encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [2106]:
maxlen = 31
tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train.tokens)
tokenized_train = tokenizer.texts_to_sequences(X_train.tokens)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

tokenized_test = tokenizer.texts_to_sequences(X_test.tokens)
x_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

# Our vectorized labels
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [2113]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
n_words = len(word_index) + 1
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std,(n_words, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [2114]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.001)

In [2137]:
batch_size = 128
epochs = 10
embed_size = 100

In [2138]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64, recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Nadam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [2139]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 31, 100)           1280900   
_________________________________________________________________
lstm_31 (LSTM)               (None, 31, 128)           117248    
_________________________________________________________________
lstm_32 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_31 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_32 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 33        
Total params: 1,453,829
Trainable params: 172,929
Non-trainable params: 1,280,900
_____________________________________

In [2140]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])

Epoch 1/10
49/49 [==============================] - 11s 221ms/step - loss: 0.6066 - accuracy: 0.6579 - val_loss: 0.6503 - val_accuracy: 0.6545
Epoch 2/10
49/49 [==============================] - 10s 211ms/step - loss: 0.4751 - accuracy: 0.7879 - val_loss: 0.8530 - val_accuracy: 0.7408
Epoch 3/10
49/49 [==============================] - 10s 212ms/step - loss: 0.4564 - accuracy: 0.7996 - val_loss: 0.5412 - val_accuracy: 0.7980
Epoch 4/10
49/49 [==============================] - 10s 211ms/step - loss: 0.4067 - accuracy: 0.8228 - val_loss: 0.5812 - val_accuracy: 0.6940
Epoch 5/10
49/49 [==============================] - ETA: 0s - loss: 0.4089 - accuracy: 0.8241
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
49/49 [==============================] - 10s 210ms/step - loss: 0.4089 - accuracy: 0.8241 - val_loss: 0.5693 - val_accuracy: 0.7950
Epoch 6/10
49/49 [==============================] - 10s 210ms/step - loss: 0.3507 - accuracy: 0.8552 - val_loss: 0.5005 - v

In [1689]:
vocab_size = len(token.word_index)
embeddings_dict = token.word_index

In [2152]:
words = list(embeddings_dict.keys())

In [2148]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_index.keys(), key=lambda word: spatial.distance.euclidean(embeddings_index[word], embedding))

In [2149]:
find_closest_embeddings(embeddings_index["earthquake"])

['earthquake',
 'quake',
 'tsunami',
 'temblor',
 'magnitude',
 'disaster',
 'aftershock',
 'earthquakes',
 'tremor',
 'quakes',
 'tsunamis',
 'jolted',
 'devastated',
 'devastating',
 'epicenter',
 'aftershocks',
 'disasters',
 'sumatra',
 'devastation',
 'richter',
 'floods',
 'struck',
 'tremors',
 'katrina',
 'flood',
 'flooding',
 'storm',
 'damage',
 'undersea',
 'measuring',
 'mudslide',
 'cyclone',
 'rocked',
 'aftermath',
 'jolts',
 'rattled',
 'magnitude-6',
 'catastrophe',
 'affected',
 'sichuan',
 'scale',
 'stricken',
 'explosion',
 'typhoon',
 'jolt',
 'calamity',
 'triggered',
 'epicentre',
 'blast',
 'ravaged',
 'wenchuan',
 'tragedy',
 'catastrophic',
 'shook',
 'storms',
 'occurred',
 'morakot',
 '7.6',
 'waves',
 'seismologists',
 '6.8',
 'seismic',
 '7.2',
 '7.2-magnitude',
 'hurricane',
 'magnitude-5',
 'tidal',
 'magnitude-7',
 'damaged',
 'tornado',
 '6.2',
 'spill',
 'temblors',
 'massive',
 'tangshan',
 '7.6-magnitude',
 'rainstorm',
 'landslides',
 'rattles',


In [2155]:

tsne = TSNE(n_components=2, random_state=0)
words =  list(embeddings_dict.values())
for word in words:
    vectors = embeddings_dict[word]
Y = tsne.fit_transform(vectors[:1000])
plt.scatter(Y[:, 0], Y[:, 1])

for label, x, y in zip(words, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")
plt.show()

KeyError: 1

In [1140]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [1141]:
# pretrained model
model_pt = Word2Vec(glove_vectors,
                     window=3,
                     vector_size=50,
                     alpha=0.1, 
                     min_alpha=0.001, 
                     workers=cores-1)
vocab = model_pt.build_vocab(texts)

model_pt.train(X_train, total_examples=len(X_train), epochs=10)

NameError: name 'glove_vectors' is not defined

In [ ]:
model_pt.wv.most_similar(positive="earthquake")

pretrained_w2v_model.save("w2v_pt.model")
model_pt = Word2Vec.load("w2v_pt.model")
word_vectors_pt = model_pt.wv
word_vectors_pt.save("w2v_pt_wv.wordvectors")

In [ ]:
model_pt.train(train_X, total_examples=pretrained_w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
wv = KeyedVectors.load("w2v_pt_wv.wordvectors", mmap='r')

In [ ]:
wv

In [1166]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC

In [1167]:

# the models that you want to compare
models = {

    'clf_knn': KNeighborsClassifier(),
    'clf_svm': SVC(),
    #'clf_xgb': XGBClassifier(),
    #'clf_ada': AdaBoostClassifier(),
 
}

# the optimisation parameters for each of the above models
params = {
#     'clf_randforest':{ 
#         'n_jobs': [-1],
#         'max_features': ['sqrt'],
#         'criterion': ['entropy'],
#         'penalty': [10, 25, 36, 64],
       
#          },
    
    'clf_knn': {
        'weights': [ 'distance'],
        'algorithm': ['ball_tree', 'kd_tree']
         },
    

    

    'clf_svm' : {
        'kernel': ['rbf', 'sigmoid'], 
     
        
         'C': [.01, .1, 1, 10, 100, 1000],
          },
    
#     'clf_xgb' : {
#         'objective': ['binary:logistic'],
#         'learning_rate': [0.001, 0.05, 0.1],
#         'alpha': [0.001, 0.3, 0.05]
#         },
        

                   
                 }

In [ ]:
def fit_search(X_data, y_data):
        """
        fits the list of models to the training data, thereby obtaining in each 
        case an evaluation score after GridSearchCV cross-validation
        """
        for name in models.keys():
            est = models[name]
            est_params = params[name]
            gscv = GridSearchCV(estimator=est, param_grid=est_params, cv=10)
            gscv.fit(X_data, y_data)
            print("best parameters are: {}".format(gscv.best_estimator_))
            y_pred = gscv.predict(X_data)
            print(classification_report(y_data, y_pred))

In [ ]:

import warnings
warnings.filterwarnings('ignore') 
fit_search(X_train, y_train)

In [ ]:
logreg = LogisticRegression(n_jobs=1, C=1e3)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

In [ ]:
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

In [ ]:
tweets_df

## train, test = train_test_split(train_df, random_state=42, test_size=0.2)

In [1099]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import Counter

In [1092]:
# verify shape of data
print('Test label shape:', np.shape(y_test))
print('Train label shape:', np.shape(y_train))

print('Test shape:', np.shape(X_test))
print('Train shape:', np.shape(X_train))

Test label shape: (683,)
Train label shape: (6147,)
Test shape: (683,)
Train shape: (6147,)


In [1120]:
from nltk.corpus import stopwords
def tokenize_text_sw(text):
    tokens = []
    stopword_list =  list(stopwords.words('english'))
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word)>1 :
                tokens.append(word.lower())
    return tokens

texts = train_df.tweets.astype(str).apply(tokenize_text)
texts

0       [our, deeds, are, the, reason, of, this, earth...
1           [forest, fire, near, la, ronge, sask, canada]
2       [all, residents, asked, to, shelter, in, place...
3       [people, receive, wildfires, evacuation, order...
4       [just, got, sent, this, photo, from, ruby, ala...
                              ...                        
7608    [two, giant, cranes, holding, bridge, collapse...
7609    [the, out, of, control, wild, fires, in, calif...
7610                       [utc, km, of, volcano, hawaii]
7611    [police, investigating, after, an, bike, colli...
7612    [the, latest, more, homes, razed, by, northern...
Name: tweets, Length: 6830, dtype: object

In [1121]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

In [1125]:
train_X = str(texts)
train_y = X_test
# convert target data with labelencoder
train_y= train_y.astype('category')
le = LabelEncoder()   
y = np.array(train_y)

le.fit_transform(y)
y = y.ravel()
# define independent variable
X = train_X
#X = X.reshape(-1,1)
# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                     test_size=.1, stratify=y)


TypeError: unhashable type: 'list'

In [1123]:
#(ngram_range=(1,2)
pipeline = Pipeline([ ('vect', CountVectorizer(ngram_range=(1,2))), 
                     ('tfidf', TfidfTransformer(use_idf=True))])
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)



counter = Counter(y_train)
print(counter)

#assign variable to np.shape tuple, width dimension
input_dimension = np.shape(X_train)
count_terms = input_dimension[1]
print(count_terms)

AttributeError: 'list' object has no attribute 'lower'

In [1151]:
vocab_size = 10000  # Only consider the top 20k words
num_tokens_per_example = 15  # Only consider the first 200 words of each movie review
X_train = keras.preprocessing.sequence.pad_sequences(
    X_train, maxlen=num_tokens_per_example
)
x_val = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=num_tokens_per_example) 

ValueError: invalid literal for int() with base 10: 'coming'

In [1150]:
from tensorflow import keras
